<a href="https://colab.research.google.com/github/anacasicande/Modelamiento/blob/main/tarea_2_Jacobo_feldman.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [25]:
import sympy as sp  # Importo sympy para poder derivar
import numpy as np  # Importo numpy para poder evaluar


P = 9.4573  # Presión en bar
T = 350.0   # Temperatura en K
R = 0.08314  # Constante de los gases en L·bar/mol·K

# Propiedades críticas del n-butano
Tc = 425.2  # Temperatura crítica en K
Pc = 37.96  # Presión crítica en bar
tr = T / Tc  # Temperatura reducida
pr = P / Pc  # Presión reducida
Z = sp.Symbol('Z')  # Creo una variable simbolica para poder derivar usando la funcion diff

# Parámetros de la ecuación de Redlich-Kwong
Ω = 0.08664  # Parámetro de la ecuación de Redlich-Kwong dado por el libro, tabla 3.1
Ψ = 0.42748  # Parámetro de la ecuación de Redlich-Kwong dado por el libro, t=abla 3.1
q = ( Ψ / Ω) * (tr**(-3/2))  #  Formula de q que usa los valores Ω y Ψ dada por el libro, ecuacion 3.51
b = ((Ω * pr )/ tr)  # Formula de b que usa los valores Ω dada por el libro, ecuacion 3.50

# Función para Z_vapor usando Redlich-Kwong
def Rk_vapor(Z):  # Defino Rk_vapor como una funcion que recibe de parametros un valor numerico
    return Z - (1 + b - q * b * ((Z - b) / (Z * (Z + b))))  # Retorna la ecuacion de Rk_vapor

# Derivada de la función Z_vapor
def dRk_vapor(Z_value):  # Defino dRk_vapor como una funcion que recibe de parametros una valor numerico
    derivadaV = sp.diff(Rk_vapor(Z), Z)  # Derivo usando la funcion sp.diff que recibe como parametros la funcion a derivar y una variable simbolica
    return derivadaV.subs(Z, Z_value).evalf()  #  con subs. se cambia la variable simbolica por un valor numerico, "evalf" es para evaluar la expresion simbolica y obtener un valor numerico y al final retorna la derivada de la funcion original para ser evaluada

# Función para Z_liquid usando Redlich-Kwong
def Rk_liquid(Z):  # Defino Rk_liquid como una funcion que recibe de parametros un valor numerico
    return Z - (b + Z * (Z + b) * ((1 + b - Z) / (q * b)))  # Retorna la ecuacion de Rk_liquid

# Derivada de la función Z_liquid
def dRk_liquid(Z_value):  # Defino dRk_liquid como una funcion que recibe de parametros una valor numerico
    derivadaL = sp.diff(Rk_liquid(Z), Z)  # Derivo usando la funcion sp.diff que recibe como parametros la funcion a derivar y una variable simbolica
    return derivadaL.subs(Z, Z_value).evalf()  #  con subs. se cambia la variable simbolica por un valor numerico, "evalf" es para evaluar la expresion simbolica y obtener un valor numerico y al final retorna la derivada de la funcion original para ser evaluada

# Implementación de Newton-Raphson
def newton_raphson(func, func_prime, Z, tol=1e-10, max_iter=200):  # Defino newton_raphson como una funcion que recibe una funcion, la derivada de esta y un valor inicial. Tambien se ponen como parametros la tolerancia y el numero de iteraciones maximas.
    x = Z  # Cambio la variable "Z" a x para mayor facilidad a la hora de escribir codigo
    error = 1  # Defino error como 1
    it = 0  # Defino iteraciones como 0 para empezar el conteo

    while error > tol and it < max_iter:  # Mientras el error sea mayor a la tolerancia y las iteraciones sean menores a las maximas iteraciones puestas el ciclo "while" seguira corriendo
        try:  # Intenta hacer lo siguiente
            fx = func(x)  # Evalua la funcion
            fpx = func_prime(x)  # Evalua la derivada de la funcion

            if abs(fpx) < tol:  # Si la derivada es menor a la tolerancia no se puede contimuar
                print("La derivada es muy pequeña, no se puede continuar")  # Mensaje de error
                return None  # Retorna None

            # Fórmula de Newton-Raphson
            x_new = x - fx / fpx  # Evalua la formula de Newton-Raphson
            error = np.abs(fx / fpx)  # Evalua el error




            # Verificar la convergencia
            if abs(x_new - x) < tol:  # Si la diferencia entre x_new y x es menor a la tolerancia se encontro la convergencia
                print(f"Convergió en la iteración {it}")  # Mensaje de convergencia
                return x_new  # Retorna el nuevo valor de x


            x = x_new  # Actualiza el valor de x
            it += 1  # Incrementa el contador de iteraciones

        except ZeroDivisionError:  # Si se produce un error de división por cero se rompe el "try"
            print("División por cero en la iteración", it)  # Mensaje de error
            return None  # Retorna None

    if it == max_iter:  # Si se llego al numero maximo de iteraciones se rompe el ciclo
        print("No converge después del número máximo de iteraciones")  # Mensaje de error
        return None  # Retorna None
    else:  # Si no se rompe el ciclo se retorna el valor de x
        return x  # Retorna el valor de x





# Estimaciones iniciales para los factores de compresibilidad Z
Z_guess_vapor = 1  # # El libro de Smith, Van Ness sugiere inicializar Z en 1 para vapor saturado.
Z_guess_liquid = 0.005  # Aproximación para el líquido

# Calcular Z usando Newton-Raphson
Z_vapor = newton_raphson(Rk_vapor, dRk_vapor, Z_guess_vapor)  # Evalua la funcion newton_raphson con la funcion Rk_vapor, la derivada de esta y la estimacion inicial para encontrar la Z de vapor
Z_liquid = newton_raphson(Rk_liquid, dRk_liquid, Z_guess_liquid)  # Evalua la funcion newton_raphson con la funcion Rk_liquid, la derivada de esta y la estimacion inicial para encontrar la Z de liquid

# Convertir Z a volúmenes molares Vm (en L/mol) y luego a cm^3/mol
Vm_vapor_cm3 = Z_vapor * R * T / P*1000  if Z_vapor else None  # Evalua la formula para encontrar el volumen molar del vapor si Z_vapor es diferente a "None"
Vm_liquid_cm3 = Z_liquid * R * T / P*1000   if Z_liquid else None  # Evalua la formula para encontrar el volumen molar del liquido si Z_liquid es diferente a "None"
print(f"Z_vapor: {Z_vapor}, Vm_vapor_cm3 = {Vm_vapor_cm3} cm^3/mol")  # Imprime el valor de Z_vapor y Vm_vapor_cm3
print(f"Z_liquid: {Z_liquid}, Vm_liquid_cm3 = {Vm_liquid_cm3} cm^3/mol")  # Imprime el valor de Z_liquid y Vm_liquid_cm3




Convergió en la iteración 3
Convergió en la iteración 5
Z_vapor: 0.830353561570580, Vm_vapor_cm3 = 2554.90026626440 cm^3/mol
Z_liquid: 0.0433061372400762, Vm_liquid_cm3 = 133.247891845345 cm^3/mol
